In [1]:
%%writefile decor_header.py
import random; from IPython.display import display,HTML
from IPython.core.magic import register_line_magic
@register_line_magic
def decor_header(params):
    params=params.split('|'); string=params[0]
    if len(params)==1: 
        font_size='22'; font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==2: 
        font_size=params[1]
        font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==3: 
        font_size=params[1]; font_family=params[2]
        cmap='Sinebow'
    else: 
        font_size=params[1]; font_family=params[2]; cmap=params[3]
    height=max([int(font_size)*2.5,60]) 
    randi=str(random.uniform(0,9999999))
    html_str="""
<script src='https://d3js.org/d3.v6.min.js'></script>
<style>
@import 'https://fonts.googleapis.com/css?family="""+font_family+"""';
#colorized001 {
font-family:"""+font_family+"""; font-size:"""+font_size+""";}
#canvas001,#canvas002 {width:10%; vertical-align:middle;}
</style>
<text id='colorized001'><canvas id='canvas001'></canvas>
"""+string+"""
<canvas id='canvas002'></canvas></text><br/>
<script>
var tc=setInterval(function() {
    var now=(new Date().getTime()%5000)/5000;
    var now_slow=(new Date().getTime()%100000)/100000;
    var iddoc=document.getElementById('colorized001');
    iddoc.style.color=d3.interpolate"""+cmap+"""(now);
    var r=10,n=7;
    var c1=document.getElementById('canvas001'); 
    var context1=c1.getContext('2d');
    var c2=document.getElementById('canvas002'); 
    var context2=c2.getContext('2d');
    c1.style.background=d3.interpolate"""+cmap+"""(now_slow); 
    c2.style.background=d3.interpolate"""+cmap+"""(now_slow);
    context1.strokeStyle=d3.interpolate"""+cmap+"""(now);    
    context2.strokeStyle=d3.interpolate"""+cmap+"""(now);
    for (var i=1; i<n; i++) {
        context1.beginPath(); context2.beginPath();
        for (var j=0; j<6; j++) {
            context1.arc(60*j,r*(n+.5),i*r,0,2*Math.PI);
            context2.arc(60*j,r*(n+.5),i*r,0,2*Math.PI); };
        context1.stroke(); context2.stroke(); }; },1)
</script>"""
    file='d3header'+randi+'.html'
    with open(file,'w') as f:
         f.write(html_str); f.close()
    string="""<div id='html_string"""+randi+\
    """' style='width:100%;'><iframe src='"""+file+\
    """' height="""+str(height)+"""
    style='display:block; width:100%;'></iframe></div>"""
    display(HTML(string))

Writing decor_header.py


In [2]:
%run decor_header.py
%decor_header Modules, Helpful Functions, & Styling

In [3]:
!pip install --upgrade neural_structured_learning --user --quiet 

In [4]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Ewert|Roboto&effect=3d');
span {text-shadow:3px 3px 3px #aaa;}  
div.output_area pre{font-family:Roboto; font-size:110%; color:#36ff36;}      
</style>

In [5]:
import warnings; warnings.filterwarnings('ignore')
import h5py,pylab as pl,pandas as pd
import numpy as np,tensorflow as tf
import neural_structured_learning as nsl
from sklearn.model_selection import train_test_split
fpath3='../input/classification-of-handwritten-letters/'
fpath4='../input/flower-color-images/'

In [6]:
def prepro(x_train,y_train,x_test,y_test):
    n=int(len(x_test)/2)    
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [y_train.shape,y_valid.shape,y_test.shape]],
                    columns=['train','valid','test'],
                    index=['image arrays','label arrays'])
    display(df)
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test]]

In [7]:
%decor_header Data Loading & Preprocessing

In [8]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.mnist.load_data()
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1]]=\
prepro(x_train1/255,y_train1.reshape(-1,1),
       x_test1/255,y_test1.reshape(-1,1))

11493376/11490434 [==============================] - 0s 0us/step


,train,valid,test
image arrays,"(60000, 28, 28)","(5000, 28, 28)","(5000, 28, 28)"
label arrays,"(60000, 1)","(5000, 1)","(5000, 1)"


In [9]:
(x_train2,y_train2),(x_test2,y_test2)=\
tf.keras.datasets.cifar10.load_data()
[[x_train2,x_valid2,x_test2],
 [y_train2,y_valid2,y_test2]]=\
prepro(x_train2/255,y_train2,x_test2/255,y_test2)

170500096/170498071 [==============================] - 4s 0us/step


,train,valid,test
image arrays,"(50000, 32, 32, 3)","(5000, 32, 32, 3)","(5000, 32, 32, 3)"
label arrays,"(50000, 1)","(5000, 1)","(5000, 1)"


In [10]:
f=h5py.File(fpath3+'LetterColorImages_123.h5','r') 
keys=list(f.keys()); print(keys)
images=np.array(f[keys[1]]).astype('float32')/255
labels=np.array(f[keys[2]]).astype('int32').reshape(-1,1)-1
x_train3,x_test3,y_train3,y_test3=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train3,x_valid3,x_test3],
 [y_train3,y_valid3,y_test3]]=\
prepro(x_train3,y_train3,x_test3,y_test3)

['backgrounds', 'images', 'labels']


,train,valid,test
image arrays,"(11352, 32, 32, 3)","(1419, 32, 32, 3)","(1419, 32, 32, 3)"
label arrays,"(11352, 1)","(1419, 1)","(1419, 1)"


In [11]:
f=h5py.File(fpath4+'FlowerColorImages.h5','r') 
keys=list(f.keys()); print(keys)
images=np.array(f[keys[0]]).astype('float32')/255
labels=np.array(f[keys[1]]).astype('int32').reshape(-1,1)
x_train4,x_test4,y_train4,y_test4=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train4,x_valid4,x_test4],
 [y_train4,y_valid4,y_test4]]=\
prepro(x_train4,y_train4,x_test4,y_test4)

['images', 'labels']


,train,valid,test
image arrays,"(168, 128, 128, 3)","(21, 128, 128, 3)","(21, 128, 128, 3)"
label arrays,"(168, 1)","(21, 1)","(21, 1)"


In [12]:
%decor_header Models with Adversarial Regularization
%decor_header MLP Base|18

In [13]:
batch_size=64; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size),name='input'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(n_class,activation=tf.nn.softmax)
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [14]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1,'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1,'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 938 steps, validate for 78 steps
Epoch 1/7
938/938 - 8s - loss: 0.2680 - sparse_categorical_crossentropy: 0.2159 - sparse_categorical_accuracy: 0.9357 - scaled_adversarial_loss: 0.0521 - val_loss: 0.2518 - val_sparse_categorical_crossentropy: 0.2031 - val_sparse_categorical_accuracy: 0.9339 - val_scaled_adversarial_loss: 0.0488
Epoch 2/7
938/938 - 6s - loss: 0.1134 - sparse_categorical_crossentropy: 0.0893 - sparse_categorical_accuracy: 0.9728 - scaled_adversarial_loss: 0.0241 - val_loss: 0.1990 - val_sparse_categorical_crossentropy: 0.1600 - val_sparse_categorical_accuracy: 0.9511 - val_scaled_adversarial_loss: 0.0390
Epoch 3/7
938/938 - 5s - loss: 0.0738 - sparse_categorical_crossentropy: 0.0569 - sparse_categorical_accuracy: 0.9825 - scaled_adversarial_loss: 0.0169 - val_loss: 0.1749 - val_sparse_categorical_crossentropy: 0.1402 - val_sparse_categorical_accuracy: 0.9615 - val_scaled_adversarial_loss: 0.0347
Epoch 4/7
938/938 - 5s - loss: 0.0501 - sparse_categorical_crossen

In [15]:
adv_model.evaluate({'input':x_test1,'label':y_test1})

5000/5000 [==============================] - 1s 257us/sample - loss: 0.0693 - sparse_categorical_crossentropy: 0.0546 - sparse_categorical_accuracy: 0.9850 - scaled_adversarial_loss: 0.0143


[0.06931666927303158, 0.05463704, 0.985, 0.014349952]

In [16]:
batch_size=64; img_size=32; n_class=10; epochs=7
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [17]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 782 steps, validate for 78 steps
Epoch 1/7
782/782 - 10s - loss: 2.1245 - sparse_categorical_crossentropy: 1.7583 - sparse_categorical_accuracy: 0.3853 - scaled_adversarial_loss: 0.3662 - val_loss: 1.9538 - val_sparse_categorical_crossentropy: 1.6201 - val_sparse_categorical_accuracy: 0.4167 - val_scaled_adversarial_loss: 0.3337
Epoch 2/7
782/782 - 7s - loss: 1.7853 - sparse_categorical_crossentropy: 1.4776 - sparse_categorical_accuracy: 0.4734 - scaled_adversarial_loss: 0.3077 - val_loss: 1.9593 - val_sparse_categorical_crossentropy: 1.6221 - val_sparse_categorical_accuracy: 0.4309 - val_scaled_adversarial_loss: 0.3372
Epoch 3/7
782/782 - 8s - loss: 1.6633 - sparse_categorical_crossentropy: 1.3749 - sparse_categorical_accuracy: 0.5102 - scaled_adversarial_loss: 0.2884 - val_loss: 1.8955 - val_sparse_categorical_crossentropy: 1.5674 - val_sparse_categorical_accuracy: 0.4513 - val_scaled_adversarial_loss: 0.3281
Epoch 4/7
782/782 - 8s - loss: 1.5402 - sparse_categorical_crosse

In [18]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

5000/5000 [==============================] - 1s 213us/sample - loss: 2.5591 - sparse_categorical_crossentropy: 2.1048 - sparse_categorical_accuracy: 0.4166 - scaled_adversarial_loss: 0.4543


[2.559125650024414, 2.1048405, 0.4166, 0.45433268]

In [19]:
batch_size=128; img_size=32; n_class=33; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(33,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [20]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 89 steps, validate for 11 steps
Epoch 1/100
89/89 - 4s - loss: 3.3316 - sparse_categorical_crossentropy: 2.7089 - sparse_categorical_accuracy: 0.2418 - scaled_adversarial_loss: 0.6227 - val_loss: 4.8022 - val_sparse_categorical_crossentropy: 3.9855 - val_sparse_categorical_accuracy: 0.0632 - val_scaled_adversarial_loss: 0.8167
Epoch 2/100
89/89 - 1s - loss: 2.4645 - sparse_categorical_crossentropy: 1.9835 - sparse_categorical_accuracy: 0.4189 - scaled_adversarial_loss: 0.4810 - val_loss: 7.9680 - val_sparse_categorical_crossentropy: 6.5944 - val_sparse_categorical_accuracy: 0.0639 - val_scaled_adversarial_loss: 1.3735
Epoch 3/100
89/89 - 1s - loss: 2.0519 - sparse_categorical_crossentropy: 1.6394 - sparse_categorical_accuracy: 0.5157 - scaled_adversarial_loss: 0.4125 - val_loss: 9.1854 - val_sparse_categorical_crossentropy: 7.5740 - val_sparse_categorical_accuracy: 0.0689 - val_scaled_adversarial_loss: 1.6114
Epoch 4/100
89/89 - 1s - loss: 1.7367 - sparse_categorical_crossent

In [21]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

1419/1419 [==============================] - 1s 433us/sample - loss: 2.2758 - sparse_categorical_crossentropy: 1.7577 - sparse_categorical_accuracy: 0.7315 - scaled_adversarial_loss: 0.5656


[2.275804270985599, 1.7576756, 0.73150104, 0.56562084]

In [22]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [23]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 11 steps, validate for 1 steps
Epoch 1/100
11/11 - 3s - loss: 3.4592 - sparse_categorical_crossentropy: 2.8646 - sparse_categorical_accuracy: 0.2679 - scaled_adversarial_loss: 0.5899 - val_loss: 6.0901 - val_sparse_categorical_crossentropy: 5.0726 - val_sparse_categorical_accuracy: 0.3125 - val_scaled_adversarial_loss: 1.0175
Epoch 2/100
11/11 - 0s - loss: 1.8424 - sparse_categorical_crossentropy: 1.5256 - sparse_categorical_accuracy: 0.5298 - scaled_adversarial_loss: 0.3099 - val_loss: 5.9369 - val_sparse_categorical_crossentropy: 4.9453 - val_sparse_categorical_accuracy: 0.3750 - val_scaled_adversarial_loss: 0.9916
Epoch 3/100
11/11 - 0s - loss: 0.3816 - sparse_categorical_crossentropy: 0.3149 - sparse_categorical_accuracy: 0.9048 - scaled_adversarial_loss: 0.0656 - val_loss: 6.3651 - val_sparse_categorical_crossentropy: 5.3013 - val_sparse_categorical_accuracy: 0.2500 - val_scaled_adversarial_loss: 1.0638
Epoch 4/100
11/11 - 0s - loss: 0.1127 - sparse_categorical_crossentr

In [24]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

21/21 [==============================] - 0s 22ms/sample - loss: 2.6811 - sparse_categorical_crossentropy: 2.2279 - sparse_categorical_accuracy: 0.4762 - scaled_adversarial_loss: 0.4532


[2.681089162826538, 2.2278523, 0.47619048, 0.45323682]

In [25]:
%decor_header CNN Base|18

In [26]:
batch_size=128; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,1),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [27]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1.reshape(x_train1.shape[0],28,28,1),
     'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1.reshape(x_valid1.shape[0],28,28,1),
     'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 469 steps, validate for 39 steps
Epoch 1/7
469/469 - 10s - loss: 0.4705 - sparse_categorical_crossentropy: 0.3900 - sparse_categorical_accuracy: 0.8736 - scaled_adversarial_loss: 0.0805 - val_loss: 0.2187 - val_sparse_categorical_crossentropy: 0.1792 - val_sparse_categorical_accuracy: 0.9453 - val_scaled_adversarial_loss: 0.0394
Epoch 2/7
469/469 - 6s - loss: 0.1187 - sparse_categorical_crossentropy: 0.0978 - sparse_categorical_accuracy: 0.9717 - scaled_adversarial_loss: 0.0209 - val_loss: 0.0744 - val_sparse_categorical_crossentropy: 0.0604 - val_sparse_categorical_accuracy: 0.9806 - val_scaled_adversarial_loss: 0.0140
Epoch 3/7
469/469 - 7s - loss: 0.0827 - sparse_categorical_crossentropy: 0.0674 - sparse_categorical_accuracy: 0.9804 - scaled_adversarial_loss: 0.0153 - val_loss: 0.0671 - val_sparse_categorical_crossentropy: 0.0542 - val_sparse_categorical_accuracy: 0.9832 - val_scaled_adversarial_loss: 0.0128
Epoch 4/7
469/469 - 6s - loss: 0.0692 - sparse_categorical_crosse

In [28]:
adv_model.evaluate({'input':x_test1.reshape(x_test1.shape[0],28,28,1),
                    'label':y_test1})

5000/5000 [==============================] - 1s 214us/sample - loss: 0.0249 - sparse_categorical_crossentropy: 0.0199 - sparse_categorical_accuracy: 0.9938 - scaled_adversarial_loss: 0.0049


[0.024870124040637164, 0.019908926, 0.9938, 0.0049104947]

In [29]:
batch_size=64; img_size=32; n_class=10; epochs=20
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(96,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [30]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 782 steps, validate for 78 steps
Epoch 1/20
782/782 - 9s - loss: 1.9731 - sparse_categorical_crossentropy: 1.6389 - sparse_categorical_accuracy: 0.3945 - scaled_adversarial_loss: 0.3343 - val_loss: 1.6796 - val_sparse_categorical_crossentropy: 1.3897 - val_sparse_categorical_accuracy: 0.5058 - val_scaled_adversarial_loss: 0.2899
Epoch 2/20
782/782 - 8s - loss: 1.5975 - sparse_categorical_crossentropy: 1.3237 - sparse_categorical_accuracy: 0.5254 - scaled_adversarial_loss: 0.2739 - val_loss: 1.5049 - val_sparse_categorical_crossentropy: 1.2405 - val_sparse_categorical_accuracy: 0.5635 - val_scaled_adversarial_loss: 0.2643
Epoch 3/20
782/782 - 8s - loss: 1.4509 - sparse_categorical_crossentropy: 1.2010 - sparse_categorical_accuracy: 0.5726 - scaled_adversarial_loss: 0.2499 - val_loss: 1.3901 - val_sparse_categorical_crossentropy: 1.1430 - val_sparse_categorical_accuracy: 0.6036 - val_scaled_adversarial_loss: 0.2471
Epoch 4/20
782/782 - 8s - loss: 1.3655 - sparse_categorical_cro

In [31]:
adv_model.evaluate({'input':x_test2,'label':y_test2})

5000/5000 [==============================] - 1s 212us/sample - loss: 1.0914 - sparse_categorical_crossentropy: 0.8784 - sparse_categorical_accuracy: 0.6896 - scaled_adversarial_loss: 0.2131


[1.0914077386856078, 0.87841755, 0.6896, 0.21311697]

In [32]:
batch_size=128; img_size=32; n_class=33; epochs=150
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(33,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [33]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 89 steps, validate for 11 steps
Epoch 1/150
89/89 - 3s - loss: 4.1937 - sparse_categorical_crossentropy: 3.4947 - sparse_categorical_accuracy: 0.0283 - scaled_adversarial_loss: 0.6990 - val_loss: 4.1836 - val_sparse_categorical_crossentropy: 3.4862 - val_sparse_categorical_accuracy: 0.0355 - val_scaled_adversarial_loss: 0.6975
Epoch 2/150
89/89 - 1s - loss: 4.1413 - sparse_categorical_crossentropy: 3.4506 - sparse_categorical_accuracy: 0.0483 - scaled_adversarial_loss: 0.6907 - val_loss: 4.0710 - val_sparse_categorical_crossentropy: 3.3907 - val_sparse_categorical_accuracy: 0.0561 - val_scaled_adversarial_loss: 0.6803
Epoch 3/150
89/89 - 1s - loss: 4.0365 - sparse_categorical_crossentropy: 3.3615 - sparse_categorical_accuracy: 0.0681 - scaled_adversarial_loss: 0.6750 - val_loss: 4.0231 - val_sparse_categorical_crossentropy: 3.3501 - val_sparse_categorical_accuracy: 0.0817 - val_scaled_adversarial_loss: 0.6730
Epoch 4/150
89/89 - 1s - loss: 3.9493 - sparse_categorical_crossent

In [34]:
adv_model.evaluate({'input':x_test3,'label':y_test3})

1419/1419 [==============================] - 1s 408us/sample - loss: 0.4753 - sparse_categorical_crossentropy: 0.3455 - sparse_categorical_accuracy: 0.8992 - scaled_adversarial_loss: 0.1330


[0.4753105203562677, 0.34550288, 0.8992248, 0.13298188]

In [35]:
batch_size=16; img_size=32; n_class=10; epochs=100
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [36]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,epochs=epochs)

Train for 11 steps, validate for 1 steps
Epoch 1/100
11/11 - 2s - loss: 2.8674 - sparse_categorical_crossentropy: 2.3886 - sparse_categorical_accuracy: 0.0893 - scaled_adversarial_loss: 0.4786 - val_loss: 2.7449 - val_sparse_categorical_crossentropy: 2.2869 - val_sparse_categorical_accuracy: 0.1250 - val_scaled_adversarial_loss: 0.4580
Epoch 2/100
11/11 - 0s - loss: 2.6709 - sparse_categorical_crossentropy: 2.2258 - sparse_categorical_accuracy: 0.2202 - scaled_adversarial_loss: 0.4453 - val_loss: 2.6441 - val_sparse_categorical_crossentropy: 2.2027 - val_sparse_categorical_accuracy: 0.1875 - val_scaled_adversarial_loss: 0.4413
Epoch 3/100
11/11 - 0s - loss: 2.4692 - sparse_categorical_crossentropy: 2.0609 - sparse_categorical_accuracy: 0.2679 - scaled_adversarial_loss: 0.4089 - val_loss: 2.5896 - val_sparse_categorical_crossentropy: 2.1569 - val_sparse_categorical_accuracy: 0.3750 - val_scaled_adversarial_loss: 0.4327
Epoch 4/100
11/11 - 0s - loss: 2.1960 - sparse_categorical_crossentr

In [37]:
adv_model.evaluate({'input':x_test4,'label':y_test4})

21/21 [==============================] - 0s 21ms/sample - loss: 1.0055 - sparse_categorical_crossentropy: 0.8260 - sparse_categorical_accuracy: 0.8571 - scaled_adversarial_loss: 0.1795


[1.0054844617843628, 0.82601565, 0.85714287, 0.17946881]